In [43]:
import pickle as pkl
import numpy as np

In [44]:
input_dim = 320

In [45]:
with open('Train_'+str(input_dim)+'.pkl', 'rb') as tr:
    X_train = pkl.load(tr)
tr.close()
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_train.shape

(87719, 320, 1)

In [48]:
with open('Val_'+str(input_dim)+'.pkl', 'rb') as val:
    X_val = pkl.load(val)
val.close()
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], 1)
X_val.shape

(998, 320, 1)

In [49]:
with open('Test_'+str(input_dim)+'.pkl', 'rb') as test:
    X_test = pkl.load(test)
test.close()
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
X_test.shape

(999, 320, 1)

In [50]:
with open('Train_labels.txt', 'r') as l:
    train_labels = l.read().split('\n')
l.close()
len(train_labels)

87719

In [51]:
with open('Val_labels.txt', 'r') as v:
    val_labels = v.read().split('\n')
v.close()
len(val_labels)

998

In [52]:
with open('Test_labels.txt', 'r') as t:
    test_labels = t.read().split('\n')
t.close()
len(test_labels)

999

In [53]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
enc.fit(train_labels)

LabelEncoder()

In [54]:
y_train_labels = enc.transform(train_labels)
y_train_labels.shape

(87719,)

In [55]:
y_val_labels = enc.transform(val_labels)
y_val_labels.shape

(998,)

In [56]:
y_test_labels = enc.transform(test_labels)
y_test_labels.shape

(999,)

In [57]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D
from keras.layers.advanced_activations import LeakyReLU
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import EarlyStopping

In [58]:
y_train = to_categorical(y_train_labels)
y_train.shape

(87719, 10)

In [59]:
y_val = to_categorical(y_val_labels)
y_val.shape

(998, 10)

In [60]:
y_test = to_categorical(y_test_labels)
y_test.shape

(999, 10)

In [61]:
from sklearn.utils.class_weight import compute_class_weight

In [62]:
# Instantiate the label encoder
le = LabelEncoder()

# Fit the label encoder to our label series
le.fit(list(y_train_labels))

# Create integer based labels Series
y_integers = le.transform(list(y_train_labels))

In [63]:
class_weights = compute_class_weight('balanced', np.unique(y_train_labels), y_train_labels)
class_weights_dict = dict(zip(le.transform(list(le.classes_)), class_weights))

In [64]:
# Parameters
batch_size = 512
epochs = 100
num_classes = 10

In [92]:
model = Sequential()

model.add(Conv1D(13, kernel_size=(10),
                 input_shape=(320,1), padding='same'))
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling1D(2, padding='same'))
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

In [93]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_6 (Conv1D)            (None, 320, 13)           143       
_________________________________________________________________
leaky_re_lu_6 (LeakyReLU)    (None, 320, 13)           0         
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 160, 13)           0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 160, 13)           0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 2080)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 10)                20810     
Total params: 20,953
Trainable params: 20,953
Non-trainable params: 0
_________________________________________________________________
None


In [94]:
from keras.utils.vis_utils import plot_model
plot_model(model, to_file='model_plot cnn.png', show_shapes=True, show_layer_names=True)

In [95]:
earlystop = EarlyStopping(monitor='val_acc', min_delta = 0.0001, patience=5, verbose=1, mode='auto')
callbacks_list = [earlystop]

In [96]:
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, callbacks=callbacks_list,
          validation_data=(X_val, y_val), class_weight = class_weights_dict, verbose=1)

Train on 87719 samples, validate on 998 samples
Epoch 1/100
87719/87719 [==============================] - 32s 361us/step - loss: 1.8801 - acc: 0.5319 - val_loss: 1.4561 - val_acc: 0.5762
Epoch 2/100
87719/87719 [==============================] - 27s 304us/step - loss: 1.2903 - acc: 0.6767 - val_loss: 1.1349 - val_acc: 0.6443
Epoch 3/100
87719/87719 [==============================] - 24s 271us/step - loss: 1.0896 - acc: 0.7069 - val_loss: 0.9901 - val_acc: 0.6884
Epoch 4/100
87719/87719 [==============================] - 26s 300us/step - loss: 0.9919 - acc: 0.7244 - val_loss: 0.9298 - val_acc: 0.7014
Epoch 5/100
87719/87719 [==============================] - 26s 293us/step - loss: 0.9304 - acc: 0.7365 - val_loss: 0.8739 - val_acc: 0.7244
Epoch 6/100
87719/87719 [==============================] - 24s 275us/step - loss: 0.8826 - acc: 0.7459 - val_loss: 0.8474 - val_acc: 0.7265
Epoch 7/100
87719/87719 [==============================] - 24s 270us/step - loss: 0.8528 - acc: 0.7518 - val_los

In [97]:
scores = model.evaluate(X_val, y_val, verbose=0)
print('Val loss:', scores[0])
print('Val accuracy:', scores[1])

Val loss: 0.7198242227395694
Val accuracy: 0.7595190380761523


In [98]:
y_pred = model.predict_classes(X_val)
confusion_matrix(y_pred, y_val_labels)

array([[63,  2,  3,  1,  1,  0,  2,  6,  2,  0],
       [ 2, 67,  3,  1,  0,  0,  2,  3,  2,  4],
       [ 3,  5, 84,  3,  1,  0,  2,  2,  2,  0],
       [12,  3,  3, 84,  0,  3,  2,  4,  0,  1],
       [ 1,  3,  0,  1, 89,  2,  2,  2,  9,  4],
       [ 3,  1,  0,  2,  1, 82,  4,  5,  2,  3],
       [ 4,  6,  0,  3,  0,  4, 80,  7,  3,  3],
       [ 8,  4,  3,  1,  0,  1,  2, 61,  1,  1],
       [ 1,  5,  2,  2,  4,  1,  1,  2, 67,  3],
       [ 3,  4,  2,  2,  4,  7,  2,  7, 12, 81]], dtype=int64)

In [99]:
test_scores = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', test_scores[0])
print('Test accuracy:', test_scores[1])

Test loss: 0.8031368544912553
Test accuracy: 0.7487487487487487


In [100]:
y_pred = model.predict_classes(X_test)
confusion_matrix(y_pred, y_test_labels)

array([[70,  1,  3,  1,  0,  0,  2,  3,  0,  2],
       [ 4, 69,  2,  0,  1,  1,  0,  2,  5,  3],
       [ 6,  4, 79,  0,  0,  3,  1,  9,  1,  2],
       [ 3,  7,  3, 80,  0,  0,  1,  4,  0,  0],
       [ 1,  2,  0,  1, 84,  1,  1,  2, 12,  3],
       [ 3,  1,  1,  6,  2, 84,  5,  1,  1,  1],
       [ 8,  4,  3,  6,  1,  3, 76,  7,  0,  2],
       [ 4,  3,  2,  6,  1,  5,  5, 64,  2,  3],
       [ 0,  6,  1,  0,  3,  1,  4,  2, 64,  6],
       [ 1,  3,  6,  0,  8,  2,  5,  5, 15, 78]], dtype=int64)